In [3]:
import backtrader as bt
import yfinance as yf
from datetime import datetime

class TestStrategy(bt.Strategy):
    def next(self):
        print(f"{self.data.datetime.date(0)} | Close: {self.data.close[0]}")

data = yf.download("AAPL", start="2020-01-01", end="2025-06-01", auto_adjust=True)
data.columns = data.columns.get_level_values(0)
feed = bt.feeds.PandasData(dataname=data)

cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)   # no params, simplest
cerebro.adddata(feed)
cerebro.broker.setcash(10000)

print("Starting Portfolio Value:", cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value:", cerebro.broker.getvalue())


/var/folders/v0/tllws09j6qnc4kds9pv97y6r0000gn/T/ipykernel_89578/3847693807.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2020-01-01", end="2025-06-01")
[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000
2020-01-02 | Close: 72.53852081298828
2020-01-03 | Close: 71.83329010009766
2020-01-06 | Close: 72.40567779541016
2020-01-07 | Close: 72.06515502929688
2020-01-08 | Close: 73.22442626953125
2020-01-09 | Close: 74.7797622680664
2020-01-10 | Close: 74.94880676269531
2020-01-13 | Close: 76.55005645751953
2020-01-14 | Close: 75.51637268066406
2020-01-15 | Close: 75.19275665283203
2020-01-16 | Close: 76.13463592529297
2020-01-17 | Close: 76.97753143310547
2020-01-21 | Close: 76.45586395263672
2020-01-22 | Close: 76.72877502441406
2020-01-23 | Close: 77.09828186035156
2020-01-24 | Close: 76.87608337402344
2020-01-27 | Close: 74.61553192138672
2020-01-28 | Close: 76.7263412475586
2020-01-29 | Close: 78.3324203491211
2020-01-30 | Close: 78.21890258789062
2020-01-31 | Close: 74.75077819824219
2020-02-03 | Close: 74.54547882080078
2020-02-04 | Close: 77.00650787353516
2020-02-05 | Close: 77.63443756103516
2020-02-06 | Close: 78.54253387451172
2020-02-07 | Close: 7